## ECOSTRESS Module demostration
The objective of this notebook is to serve as an example of how to work with the ECOSTRESS module, import and use the associated functions.\
ECOSTRESSS module is meant to help the users to interact with the NASA AppEEARS API to obtain and process ECOSTRESS satellite data

### Step 0: Import from the ECOSTRESS module the require functions

In [2]:
from ECOSTRESS import *

### Step 1: Authentication - Generate a token
The NASA AppEEARS API utilizes the authentication system employed by the NASA Earthdata Login. Therefore, in order to obtain the token required for authentication, users must have an account registered on the NASA Earthdata Portal, accessible at: https://www.earthdata.nasa.gov/.

In [4]:
token= get_token('mprodriguezn15')

### Step 2: Submit a task
In this example, two requests are made to retrieve ECOSTRESS data for Salzburg, specifically for the summers of 2022 and 2023. The Salzburg geojson file used in this process was created using the website: https://geojson.io.

In [5]:
# REQUEST 1
# Name of of the request
request_name= "Salzburg_Summer_22"
# Dates of interest 'MM-DD-YYYY'
start_date = '06-01-2022'
end_date = '08-31-2022'
# Area of interest
aoi = 'AOI.geojson'
submit_task(token,request_name, start_date, end_date, aoi)

{'task_id': 'bb2beb07-c5c8-459e-9641-7b24a6bb477e', 'status': 'pending'}

In [6]:
# REQUEST 2
# Name of of the requestS
request_name= "Salzburg_Summer_23"
# Dates of interest 'MM-DD-YYYY'
start_date = '06-01-2023'
end_date = '08-31-2023'
# Area of interest
aoi = 'AOI.geojson'
submit_task(token,request_name, start_date, end_date, aoi)

{'task_id': 'f52b9c8b-8586-4684-ab5a-70eb4024ef39', 'status': 'pending'}

### Step 3: Verify the info of all the tasks requested by an user account
In the previous step, two tasks were submitted using the same user account. However, it is important to note that additional requests have been made by the same user before, as confirmed in the subsequent cell.

In [4]:
list_tasks(token)

[{'name': 'Salzburg_Summer_23',
  'id': 'f52b9c8b-8586-4684-ab5a-70eb4024ef39',
  'status': 'done'},
 {'name': 'Salzburg_Summer_22',
  'id': 'bb2beb07-c5c8-459e-9641-7b24a6bb477e',
  'status': 'done'},
 {'name': 'Salzburg_Winter_18_19',
  'id': '1fedd3ed-a973-41b0-b353-22eeb02d6eac',
  'status': 'expired'},
 {'name': 'Salzburg_Winter_19_20',
  'id': '61879f9c-8b6e-4c27-8575-dd57b5b74cc5',
  'status': 'expired'},
 {'name': 'Salzburg_Winter_20_21',
  'id': 'fc7a007e-9e5d-4160-9468-c0d4a0d61f4c',
  'status': 'expired'},
 {'name': 'Salzburg_Winter_21_22',
  'id': '00dccee7-3043-48ff-84c1-af654d722df4',
  'status': 'expired'},
 {'name': 'Salzburg_Winter_22_23',
  'id': '650c6e3a-6893-4157-b42c-b1dd1d12eb41',
  'status': 'expired'}]

### Step 4: Download the data bundles of only the tasks of interest
Although multiple tasks may exist for the user account, in this case, only the data bundles for the Summer of 2022 and 2023 are being downloaded. The files within these data bundles are being saved into separate folders named "Salzburg_Summer_22_Raw" and "Salzburg_Summer_23_Raw" respectively.

In [5]:
list_task_id=['bb2beb07-c5c8-459e-9641-7b24a6bb477e', 'f52b9c8b-8586-4684-ab5a-70eb4024ef39']
download_data_bundles(token, list_task_id)

Download for Task ID bb2beb07-c5c8-459e-9641-7b24a6bb477e corresponding to Task Name Salzburg_Summer_22 has been completed.
Download for Task ID f52b9c8b-8586-4684-ab5a-70eb4024ef39 corresponding to Task Name Salzburg_Summer_23 has been completed.


### Step 5: Cloud mask the Ecostress raw images based on cloud files
Now that all the raw images and their corresponding cloud mask files are stored in the "Salzburg_Summer_22_Raw" and "Salzburg_Summer_23_Raw" folders, these folders will be used as the input directories for the cloud masking process. The resulting cloud-masked TIFF images will be saved in new folders named "Salzburg_Summer_22_Masked" and "Salzburg_Summer_23_Masked" respectively.

In [6]:
list_folder_name=['Salzburg_Summer_22_Raw', 'Salzburg_Summer_23_Raw']
apply_cloud_masking(list_folder_name)

Salzburg_Summer_22_Raw\ECO2LSTE.001_SDS_LST_doy2022152141303_aid0001.tif
Masking for folder Salzburg_Summer_22_Raw has been completed.
Salzburg_Summer_23_Raw\ECO2LSTE.001_SDS_LST_doy2023154124542_aid0001.tif
Masking for folder Salzburg_Summer_23_Raw has been completed.


### Step 6: Create Summer Median Composites
In the folders "Salzburg_Summer_22_Masked" and "Salzburg_Summer_23_Masked," all Ecostress Images captured during the Summer periods of 2022 and 2023, respectively, are stored with applied cloud masking. Now two summer median composites are created, one for each folder, and stored in a new folder called "Summer Median Composites". This process aims to reduce noise and generate cleaner images that provide representative information about the whole summer period. The median statistic is selected due to its reduced sensitivity to outliers, offering valuable insights into the behavior of Land Surface Temperature during the summer season.

In [7]:
list_folder_name=['Salzburg_Summer_22_Masked','Salzburg_Summer_23_Masked']
create_summer_median_composite(list_folder_name)

Summer Median Composite for images in the folder Salzburg_Summer_22_Masked has been completed.
Summer Median Composite for images in the folder Salzburg_Summer_23_Masked has been completed.


### Step 7: Format Summer Median Composites according to the Cloud Optimized GeoTIFF (COG) best practices
But, What are Cloud Optimized GeoTIFFs (COGs)?

A Cloud Optimized GeoTIFF (COG) is a GeoTIFF file with an internal organization that enables more efficient workflows in the cloud environment.  It does this by leveraging the ability of clients issuing ​HTTP GET range requests to ask for just the parts of a file they need [Source: What are Cloud Optimized GeoTIFFs (COGs)? | U.S. Geological Survey (usgs.gov)](https://www.usgs.gov/faqs/what-are-cloud-optimized-geotiffs-cogs#:~:text=A%20Cloud%20Optimized%20GeoTIFF%20(COG,of%20a%20file%20they%20need.))

More information can be checked in the following link: [Provider's Guide to COG (cogeo.org)](https://www.cogeo.org/providers-guide.html)

The "Summer Median Composites folder" currently contains the composite images in TIFF format, which serves as the input source for converting all of them to the COG (Cloud-Optimized GeoTIFF) format.

In [2]:
folder_name = 'Summer Median Composites'
format_median_composite_cog(folder_name)

Summer Median Composite Median_Salzburg_Summer_22_Masked.tif was successfully format to COG.
Summer Median Composite Median_Salzburg_Summer_23_Masked.tif was successfully format to COG.


### Step 8 and 9: Create Hexagons with Zonal Statistics
Please check the Example_HEXAGONS.ipynb notebook